# Подключение библиотек

In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
import time
import struct
import os

# Подключимся к MetaTrader 5

In [3]:
if not mt5.initialize():
    print("initialize() failed")
    quit()

# Подготавливаем все запросы

In [4]:
symbol_Euro = "EURUSD"
symbol_Gbp = "GBPUSD"
symbol_Euro_info =mt5.symbol_info(symbol_Euro)
symbol_Gbp_info =mt5.symbol_info(symbol_Gbp)
lot = 0.01
point_euro = mt5.symbol_info(symbol_Euro).point 
point_gbp = mt5.symbol_info(symbol_Gbp).point
deviation = 1 # deviation = отклонение
# запросы
#  struct MqlTradeRequest
#   {
#    ENUM_TRADE_REQUEST_ACTIONS    action;           // Тип выполняемого действия
#    ulong                         magic;            // Штамп эксперта (идентификатор magic number)
#    ulong                         order;            // Тикет ордера
#    string                        symbol;           // Имя торгового инструмента
#    double                        volume;           // Запрашиваемый объем сделки в лотах
#    double                        price;            // Цена 
#    double                        stoplimit;        // Уровень StopLimit ордера
#    double                        sl;               // Уровень Stop Loss ордера
#    double                        tp;               // Уровень Take Profit ордера
#    ulong                         deviation;        // Максимально приемлемое отклонение от запрашиваемой цены
#    ENUM_ORDER_TYPE               type;             // Тип ордера
#    ENUM_ORDER_TYPE_FILLING       type_filling;     // Тип ордера по исполнению
#    ENUM_ORDER_TYPE_TIME          type_time;        // Тип ордера по времени действия
#    datetime                      expiration;       // Срок истечения ордера (для ордеров типа ORDER_TIME_SPECIFIED)
#    string                        comment;          // Комментарий к ордеру
#    ulong                         position;         // Тикет позиции
#    ulong                         position_by;      // Тикет встречной позиции
#   };
request_euro_sell = {
"action": mt5.TRADE_ACTION_DEAL,# Совершить сделку прямо сейчас
"symbol": symbol_Euro, # Сделка EURRUB
"volume": lot, # Объем который мы покупаем
"type": mt5.ORDER_TYPE_SELL, # Покупаем или продаём  
#"price": price, # Для .TRADE_ACTION_DEAL не требуется
"deviation": deviation, # Максимальное отклонение от заданной цены
"magic": 123456, #Идентификатор эксперта. Позволяет организовать аналитическую обработку торговых ордеров.
"comment": "продали евро",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,   #не будет продажи если нет необходимого обьема 
    
                                            #Означает согласие совершить сделку 
                                            #по максимально доступному на рынке 
                                            #объему в пределах указанного в ордере. 
                                            #В случае невозможности полного исполнения ордер 
                                            #будет исполнен на доступный объем, а неисполненный объем ордера будет отменен.
}
request_gbp_buy= {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_Gbp, # Сделка GBPRUB
"volume": lot,
"type": mt5.ORDER_TYPE_BUY, 
"deviation": deviation,
"magic": 123456, #Идентификатор эксперта. Позволяет организовать аналитическую обработку торговых ордеров.
"comment": "купили евро",
"type_time": mt5.ORDER_TIME_GTC,        # Ордер будет находиться в очереди до тех пор, пока не будет снят
"type_filling": mt5.ORDER_FILLING_FOK,      
}

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
order_euro_sell = mt5.order_send(request_euro_sell)
order_gbp_buy = mt5.order_send(request_gbp_buy)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
request_euro_buy = { 
"action": mt5.TRADE_ACTION_DEAL, 
"symbol": symbol_Euro, 
"volume": lot, 
"type": mt5.ORDER_TYPE_BUY,
"position": order_euro_sell.order,
"deviation": deviation, 
"magic": 123456, 
"comment": "купили евро", 
"type_time": mt5.ORDER_TIME_GTC, 
"type_filling": mt5.ORDER_FILLING_FOK, 
}
request_gbp_sell = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_Gbp, # Сделка GBPRUB
"volume": lot, # Объем который мы покупаем
"type": mt5.ORDER_TYPE_SELL,
"position": order_gbp_buy.order,
"deviation": deviation,
"magic": 123456, #Идентификатор эксперта. Позволяет организовать аналитическую обработку торговых ордеров.
"comment": "продажа фунта",
"type_time": mt5.ORDER_TIME_GTC,        # Ордер будет находиться в очереди до тех пор, пока не будет снят
"type_filling": mt5.ORDER_FILLING_FOK,   
}
request_euro_buy_all= {
"action": mt5.TRADE_ACTION_DEAL,# Совершить сделку прямо сейчас
"symbol": symbol_Euro, # Сделка EURRUB
"volume": 10.0,
"type": mt5.ORDER_TYPE_BUY, # Покупаем или продаём
"position": order_euro_sell.order,
"deviation": deviation, # Максимальное отклонение от заданной цены
"magic": 123456, #Идентификатор эксперта. Позволяет организовать аналитическую обработку торговых ордеров.
"comment": "продали доллар",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_IOC,
}
request_gbp_sell_all= {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_Gbp, # Сделка GBPRUB
"volume": 10.0,
"type": mt5.ORDER_TYPE_SELL,
"position": order_gbp_buy.order,
"deviation": deviation,
"magic": 123456, #Идентификатор эксперта. Позволяет организовать аналитическую обработку торговых ордеров.
"comment": "купили евро",
"type_time": mt5.ORDER_TIME_GTC,        # Ордер будет находиться в очереди до тех пор, пока не будет снят
"type_filling": mt5.ORDER_FILLING_IOC,
}

# Функции для открытия и закрытия позиции

In [5]:
def open_close(spread):
    #максимальное и минимальное значение
    max_spread = max(spread)
    min_spread = min(spread)
    #Найдём медиану
    spread = np.array(spread)
    median = np.median(spread)
    print('median:     ',median,
          '\nmax spread: ', max_spread,
          '\nmin_spread: ',min_spread)
    Open = median + (max_spread - median) * 0.1
    print('Open:', Open)
    Close = median #+ (min_spread - median) * 0.01
    print('Close', Close)
    return Open, Close

In [6]:
def request_open_pos(): 
    result =mt5.order_send(request_gbp_buy) 
    result2 = mt5.order_send(request_euro_sell)
    print('Открыл')

In [7]:
def request_close_pos():
    result = mt5.order_send(request_gbp_sell) 
    result2 = mt5.order_send(request_euro_buy)
    print('Закрыл')

In [8]:
def request_close_pos_all():
    result = mt5.order_send(request_gbp_sell_all) 
    result2 = mt5.order_send(request_euro_buy_all)
    print(result)
    print(result2)

In [11]:
Open = 0
Close = 0

# Бесконечный цикл 1:
## Открывает множество позиции в небольшом обеме и закрывает их когда достигнет необходимое условие(Условие формируется на тиках за последние 5 минут)

In [9]:
quantity_open = 0 #Счетчик для открытых позиций
quantity_close = 0#Счетчик для закрытых позиций
balance_old = mt5.account_info().balance # Баланс
spread1 = []#Наши исторические данные будут формироваться в этом списке
while(True):
    if datetime.now().minute % 5 == 0 and datetime.now().second == 0:
        file_obj = open('QuantityOpenClose.txt', 'a')# Файл в который мы запишем результаты
        Quant_op = 'Quantity open: ' + str(quantity_open)+'\n'
        Quant_cl = 'Quantity close: ' + str(quantity_close)+'\n'
        Money = str(mt5.account_info().balance - balance_old) + '\n'
        balance_old = mt5.account_info().balance
        file_obj.write(Quant_op)
        file_obj.write(Quant_cl)
        file_obj.write(Money)
        file_obj.close()
        request_close_pos_all()
        Open,Close = open_close(spread1)
        spread1 = []
        quantity_open = 0
        quantity_close = 0
        time.sleep(1)
    last_tick_EURUSD = mt5.symbol_info_tick("EURUSD").bid # Цену закрытия
    last_tick_GPBUSD = mt5.symbol_info_tick("GBPUSD").ask
    last_spread = last_tick_GPBUSD - last_tick_EURUSD
    spread1.append(last_spread)
    if last_spread > Open:
        quantity_open += 1
        request_open_pos()
    if last_spread < Close:
        quantity_close += 1
        request_close_pos()

NameError: name 'Open' is not defined

# Бесконечный цикл 2:
## Отрывает позицию в большом объеме и закрывает её как достигнет условие(Условие формируется на последних 20 тиках)

In [13]:
spread2 = []
a = True
quantity_open = 0
quantity_close = 0
balance_old = mt5.account_info().balance
while(True):
    if datetime.now().minute % 5 == 0 and datetime.now().second == 0:
        file_obj = open('QuantityOpenClose2.txt', 'a')
        Quant_op = 'Quantity open: ' + str(quantity_open)+'\n'
        Quant_cl = 'Quantity close: ' + str(quantity_close)+'\n'
        Money = str(mt5.account_info().balance - balance_old) + '\n'
        balance_old = mt5.account_info().balance
        file_obj.write(Quant_op)
        file_obj.write(Quant_cl)
        file_obj.write(Money)
        file_obj.close()
        time.sleep(1)
    last_tick_EURUSD = mt5.symbol_info_tick("EURUSD").bid
    last_tick_GPBUSD = mt5.symbol_info_tick("GBPUSD").ask
    last_spread = last_tick_GPBUSD - last_tick_EURUSD
    spread2.append(last_spread)
    if len(spread2) == 20:
        Open = max(spread2)
        Close = min(spread2)
        spread2 = []
    if last_spread > Open and a == True:
        a = False
        quantity_open += 1
        request_open_pos()
    if last_spread < Close and a == False:
        a = True
        quantity_close += 1
        request_close_pos()

Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл
Закрыл
Открыл

KeyboardInterrupt: 

In [108]:
mt5.shutdown()

True

#  Далее черновой код не использующийся в основной программе

In [ ]:
pip install pyinstaller

In [ ]:
pyinstaller --onefile Project2_KARTLI.ipynb

In [41]:
while(True):
    print(datetime.now())

2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.409712
2021-02-10 22:27:40.410713
2021-02-10 22:27:40.410713
2021-02-10 22:27:40.410713
2021-02-10 22:27:40.410713
2021-02-10 22:27:40.410713
2021-02-10 22:27:40.410713
2

2021-02-10 22:27:40.590724
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.669322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2021-02-10 22:27:40.670322
2

2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.885336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2021-02-10 22:27:40.886336
2

2021-02-10 22:27:41.059350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.060350
2021-02-10 22:27:41.061350
2021-02-10 22:27:41.061350
2021-02-10 22:27:41.061350
2021-02-10 22:27:41.061350
2021-02-10 22:27:41.061350
2021-02-10 22:27:41.061350
2021-02-10 22:27:41.061350
2021-02-10 22:27:41.061350
2021-02-10 22:27:41.061350
2021-02-10 22:27:41.061350
2

2021-02-10 22:27:41.252364
2021-02-10 22:27:41.252364
2021-02-10 22:27:41.252364
2021-02-10 22:27:41.252364
2021-02-10 22:27:41.252364
2021-02-10 22:27:41.252364
2021-02-10 22:27:41.252364
2021-02-10 22:27:41.252364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2021-02-10 22:27:41.253364
2

...

2021-02-10 22:27:43.770080
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.791083
2021-02-10 22:27:43.792082
2021-02-10 22:27:43.792082
2021-02-10 22:27:43.793083
2021-02-10 22:27:43.793083
2021-02-10 22:27:43.793083
2021-02-10 22:27:43.793083
2021-02-10 22:27:43.793083
2021-02-10 22:27:43.793083
2021-02-10 22:27:43.793083
2021-02-10 22:27:43.793083
2

2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.985095
2021-02-10 22:27:43.986096
2

2021-02-10 22:27:44.156109
2021-02-10 22:27:44.177113
2021-02-10 22:27:44.177113
2021-02-10 22:27:44.177113
2021-02-10 22:27:44.177113
2021-02-10 22:27:44.177113
2021-02-10 22:27:44.177113
2021-02-10 22:27:44.177113
2021-02-10 22:27:44.178113
2021-02-10 22:27:44.178113
2021-02-10 22:27:44.178113
2021-02-10 22:27:44.178113
2021-02-10 22:27:44.178113
2021-02-10 22:27:44.178113
2021-02-10 22:27:44.178113
2021-02-10 22:27:44.178113
2021-02-10 22:27:44.178113
2021-02-10 22:27:44.178113
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2021-02-10 22:27:44.179111
2

2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.368124
2021-02-10 22:27:44.369124
2021-02-10 22:27:44.369124
2021-02-10 22:27:44.369124
2021-02-10 22:27:44.369124
2021-02-10 22:27:44.369124
2021-02-10 22:27:44.369124
2021-02-10 22:27:44.369124
2021-02-10 22:27:44.369124
2021-02-10 22:27:44.369124
2021-02-10 22:27:44.369124
2021-02-10 22:27:44.369124
2021-02-10 22:27:44.369124
2

KeyboardInterrupt: 

In [113]:
last_spread

0.17162999999999995

In [ ]:
from_date=datetime.now() - timedelta(days=1)
to_date=datetime.now()
history_orders=mt5.history_orders_total(from_date, datetime.now())
if history_orders>0:
    print("Total history orders=",history_orders)
else:
    print("Orders not found in history")

In [ ]:
positions_total=mt5.positions_total()
if positions_total>0:
    print("Total positions =",positions_total)
else:
    print("Positions not found")

In [24]:

lasttick=mt5.symbol_info_tick("GBPUSD")

lasttick.bid

1.36712

In [8]:
define EXPERT_MAGIC 123456   // MagicNumber эксперта
def OnStart():
   MqlTradeRequest request={0};
   MqlTradeResult  result={0};

   request.action   =TRADE_ACTION_DEAL;                     # тип торговой операции
   request.symbol   ="GBPUSD";                              # символ
   request.volume   =0.1;                                   # объем в 0.1 лот
   request.type     =ORDER_TYPE_BUY;                        # тип ордера
   request.price    =SymbolInfoDouble(Symbol(),SYMBOL_ASK); # цена для открытия
   request.deviation=5;                                     # допустимое отклонение от цены
   request.magic    =EXPERT_MAGIC;                         # MagicNumber ордера

   if(!OrderSend(request,result))
      PrintFormat("OrderSend error %d",GetLastError());     # если отправить запрос не удалось, вывести код ошибки
   PrintFormat("retcode=%u  deal=%I64u  order=%I64u",result.retcode,result.deal,result.order);

SyntaxError: invalid syntax (<ipython-input-8-7a7daa745a3e>, line 1)

In [ ]:
PrintFormat("retcode=%u  deal=%I64u  order=%I64u",result.retcode,result.deal,result.order);

In [ ]:
def user_login():
    try:
    login(
    login,                    // номер счета
    password="PASSWORD",      // пароль
    server="SERVER",          // имя сервера, как оно задано в терминале
    timeout=TIMEOUT           // таймаут
    )
    except  TimeoutErrore

# account=25115284
# authorized=mt5.login(account, password="gqrtz0lbdm")
# user_login()

def errors():
     print("initialize() failed, error code =",mt5.last_error())
#   

In [ ]:
Class Money:
    name = "RUB"
    price
# 
Class Dollar(Money):
    price
# 

Class Euro(Money):
    price
    def __init__(self,price):
        self.price = price
    def __del__():
        print(seld.price,"удален")
# 
euro = Euro()


In [55]:
# advance_gpb = gbprub_ticks_frame['ask'][0] - gbprub_ticks_frame['ask'][-1]
# advance_euro = eurrub_ticks_frame['ask'][0] - eurrub_ticks_frame['ask'][-1]

KeyError: -1